# Start here — paste your file paths (no coding needed)

1. Download the two CSV files to your computer:
   - **Meta Ad Library export** → e.g. `meta_ad_library.csv`
   - **Official election results** → e.g. `election_results.csv`
2. Copy the *full path* of each file from your file manager.
3. Paste them into the two boxes below (between the quotes) and run the cell (Shift+Enter).

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ←← Paste your file paths here
META_ADS_PATH = r""  # e.g., r"C:\Users\You\Downloads\meta_ad_library.csv" (Windows) or "/Users/you/Downloads/meta_ad_library.csv" (macOS)
ELECTIONS_PATH = r""  # e.g., r"C:\Users\You\Downloads\election_results.csv" or "/Users/you/Downloads/election_results.csv"

def _require_csv(path_str: str, label: str) -> Path:
    if not path_str:
        raise ValueError(f"No path provided for {label}. Please paste the full path between the quotes above.")
    p = Path(path_str)
    if not p.exists():
        raise FileNotFoundError(f"{label} not found at:\n{p}\nPlease check the path.")
    if p.is_dir():
        raise IsADirectoryError(f"{label}: this is a folder, not a CSV file:\n{p}")
    return p

META_ADS_CSV = _require_csv(META_ADS_PATH, "Meta Ad Library CSV")
ELECTIONS_CSV = _require_csv(ELECTIONS_PATH, "Election Results CSV")
print("Using:\n  Meta Ad Library:", META_ADS_CSV, "\n  Election Results:", ELECTIONS_CSV)

# Outputs directory (where the notebook will save generated files)
from pathlib import Path as _P
OUT_DIR = _P("outputs"); OUT_DIR.mkdir(exist_ok=True)


## Note on saved files

Figures and tables are displayed directly here in the notebook.

At the same time, CSV files and plots are also saved into the folder **`outputs/`**,
which will appear in the same location where you saved this notebook.

You can open these CSV files in Excel if you want to explore them further.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# === USER INPUT REQUIRED ===
# Please copy/paste the path or URL of the CSV files here

META_ADS_CSV   = str(META_ADS_CSV)
ELECTIONS_CSV  = str(ELECTIONS_CSV)

# Convert to Path objects if they are local files
META_ADS_CSV   = Path(META_ADS_CSV)
ELECTIONS_CSV  = Path(ELECTIONS_CSV)

print("Meta ads file:", META_ADS_CSV)
print("Election file:", ELECTIONS_CSV)


# Meta Ad Library Analysis #

This notebook shows code used by Bellingcat to clean and analyse data from the Meta Ad Library from pages running ads sponsored by Die Linke and other affiliated pages seen by Facebook and Instagram users between January 1 and Feb. 23, 2025 in the lead up to the federal election 2025.

This code uses the following CSV files:

meta-ad-library.csv contains combined data from these advertisers/pages that ran ads sponsored by Die Linke' campaign.

results_2025_German_election.csv containing the overall results from all areas (federal territory, federal states, and constituencies) in tabular form published by the Federal Returning Officer.

It can also be repurposed to analyse other CSV files downloaded directly from the Meta Ad Library, as long as references to these files are added to data_files.

This code was originally put together by Pooja Chaudhuri and Melissa Zhu, but is adjusted by Karla Hootz for the purpose of analysing the rise of Die Linke in the federal election 2025 in Germany.

## Imports and basic functions ##

In [ ]:
pip install scikit-learn # type: ignore

In [ ]:
pip install nltk # type: ignore

In [ ]:
pip install seaborn # type: ignore

In [ ]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import nltk
import unicodedata
import os
import re

In [ ]:
# Define variables for each candidate's data; replace with appropriate filenames if replicating with other data

data_files = {
    'die_linke': str(META_ADS_CSV),
    # add or replace as needed
}

In [ ]:
def add_average_spending(df):
    '''Splits 'spend' column into its upper and lower limits, and calculates the average, adding a new column with this value'''

    df[['lower_spend', 'upper_spend']] = df['spend'].str.extract(r'lower_bound:\s*(\d+),\s*upper_bound:\s*(\d+)')

    df['lower_spend'] = pd.to_numeric(df['lower_spend'])
    df['upper_spend'] = pd.to_numeric(df['upper_spend'])

    df['spend_average'] = (df['lower_spend'] + df['upper_spend']) / 2

    print(f"The total amount spent on all Meta ads in the data is about ${df['spend_average'].sum()} based on the average of the upper and lower limits for spending on each ad.")
    return df

In [ ]:
import json 

def process_delivery_by_region(df):
    '''Takes the data in 'delivery_by_region', which is a dictionary showing the percentage of impressions from each region, and creates
    a new column for each region, with the percentage of impressions for that region as the value.'''

    rows = []

    # Parse delivery_by_region and construct a list of rows with region and percentage data
    for index, cell in df[['ad_archive_id', 'delivery_by_region']].dropna().iterrows():
        try:
            # Parse JSON-like data from the cell
            parsed_data = json.loads(f'[{cell["delivery_by_region"]}]')
        except json.JSONDecodeError as e:
            print(f"Error parsing cell: {cell['delivery_by_region']}\n{e}")
            parsed_data = []

        # Add ad_archive_id to each parsed entry
        for entry in parsed_data:
            entry['ad_archive_id'] = cell['ad_archive_id']
            rows.append(entry)

    # Create a DataFrame from the rows
    regions_df = pd.DataFrame(rows)

    # Pivot the regions DataFrame to have regions as columns
    regions_pivot = regions_df.pivot_table(
        index='ad_archive_id',
        columns='region',
        values='percentage',
        aggfunc='first'
    ).fillna(0) * 100  # Convert to percentage

    # Clean up column names and reset index
    regions_pivot.columns.name = None
    regions_pivot = regions_pivot.reset_index()

    # Merge the pivoted regions data back into the original DataFrame
    df_merged = df.merge(regions_pivot, on='ad_archive_id', how='left')

    # Remove rows with null delivery_by_region values
    df_merged = df_merged.dropna(subset=['delivery_by_region'])

    state_columns = ['Baden-W\u00fcrttemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz',
       'Sachsen', 'Saxony-Anhalt', 'Schleswig-Holstein', 'Th\u00fcringen', 'Saarland', 'Berlin (city)', 'Bremen (city)', 'Alsace']

    # Print region names
    region_names = [col for col in regions_pivot.columns if col != 'ad_archive_id']
    non_de_regions = [col for col in region_names if col not in state_columns]
    print("Region columns added:")
    print(region_names)
    print()
    print("This includes the following non-US regions:")
    print(non_de_regions)
    print()

    # Calculate the total non-US delivery percentage out of the overall total
    non_de_total_percentage = df_merged[non_de_regions].sum().sum() / df_merged[region_names].sum().sum() * 100
    print(f"Total percentage of ad delivery in non-US regions across all ads: {non_de_total_percentage:.3f}%")

    return region_names, non_de_regions, df_merged

import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(str(META_ADS_CSV))
# Optional: call and visualization
region_names, non_de_regions, df_with_regions = process_delivery_by_region(df)

# Beispiel-Plot: Durchschnittliche Anteile der Top 10 Regionen
region_means = df_with_regions[region_names].mean().sort_values(ascending=False).head(10)
region_means.plot(kind="barh")
plt.xlabel("percentage")
plt.ylabel("region")
plt.title("Top 10 regional distribution by ad")
plt.tight_layout()
plt.show()

In [ ]:
import re
import pandas as pd

# --- Mapping (from our earlier step) ---
city_to_state = {
    "Berlin": "Berlin",
    "Hamburg": "Hamburg",
    "Bielefeld": "Nordrhein-Westfalen",
    "Dresden": "Sachsen",
    "Hildesheim": "Niedersachsen",
    "Erfurt": "Thüringen",
    "Offenbach": "Hessen",
    "Leipzig": "Sachsen",
}

# Regex patterns to detect city names in text
city_patterns = {city: re.compile(rf"\b{city}\b", re.IGNORECASE) for city in city_to_state.keys()}

# --- Add boolean flags for city mentions in ad_creative_bodies ---
for city, pattern in city_patterns.items():
    df_with_regions[f"mention_{city}"] = df_with_regions["ad_creative_bodies"].astype(str).str.contains(pattern)

# --- Analysis: compare delivery in matching Bundesland ---
results = []

for city, state in city_to_state.items():
    mention_mask = df_with_regions[f"mention_{city}"]

    if state in df_with_regions.columns:
        avg_with = df_with_regions.loc[mention_mask, state].mean()
        avg_without = df_with_regions.loc[~mention_mask, state].mean()
        results.append({
            "City": city,
            "State": state,
            "Mentions": mention_mask.sum(),
            "AvgDelivery_withMention": avg_with,
            "AvgDelivery_withoutMention": avg_without,
            "Difference": avg_with - avg_without
        })

results_df = pd.DataFrame(results).sort_values("Difference", ascending=False)

print("Correlation between city mentions in text and delivery by region:")
print(results_df)

# --- Optional Plot ---
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.barh(results_df["City"], results_df["Difference"], color="steelblue")
plt.axvline(0, color="black", linestyle="--")
plt.xlabel("Difference in average share (%)")
plt.title("Greater reach in the region when the city is mentioned in the ad text")
plt.tight_layout()
plt.show()





In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import re

# Define pattern for Berlin (include district mentions if needed)
berlin_pattern = re.compile(r"\bberlin\b|\bneukölln\b", flags=re.IGNORECASE)

# Create binary indicator: does the ad mention Berlin?
df_with_regions["mention_berlin"] = df_with_regions["ad_creative_bodies"].astype(str).str.contains(berlin_pattern)

# Build DataFrame for plotting
df_plot = pd.DataFrame({
    "Delivery %": pd.to_numeric(df_with_regions["Berlin"], errors="coerce"),
    "Mention Berlin": df_with_regions["mention_berlin"].map({True: "Yes", False: "No"})
}).dropna()

# Plot boxplot
plt.figure(figsize=(6,4))
df_plot.boxplot(column="Delivery %", by="Mention Berlin", grid=False)
plt.suptitle("")
plt.title("Berlin delivery share by text mention")
plt.ylabel("Delivery (%)")
plt.tight_layout()
plt.show()

In [ ]:
# --- Build df_demographics from 'demographic_distribution' ---
def safe_json_loads(s):
    if not isinstance(s, str):
        return []
    s = s.replace("'", '"')
    if not s.strip().startswith("["):
        s = "[" + s + "]"
    try:
        return json.loads(s)
    except Exception:
        return []

df_with_regions["demographic_distribution"] = df_with_regions["demographic_distribution"].apply(safe_json_loads)

df_exploded = df_with_regions.explode("demographic_distribution")

df_demographics = pd.concat(
    [
        df_exploded[["ad_archive_id"]].reset_index(drop=True),
        pd.json_normalize(df_exploded["demographic_distribution"]).reset_index(drop=True)
    ],
    axis=1
).dropna(subset=["ad_archive_id"])


In [ ]:
# ================= Microtargeting Index (uses SAME notebook definitions) =================
import numpy as np
import pandas as pd
import re

# Utilities
def _safe_minmax(x: pd.Series) -> pd.Series:
    x = x.astype(float)
    if x.max() == x.min():
        return pd.Series(0.0, index=x.index)
    return (x - x.min()) / (x.max() - x.min())

def _hhi(shares) -> float:
    """Herfindahl over shares (expects non-negative; normalizes to sum=1)."""
    arr = np.asarray(shares, dtype=float)
    arr = arr[np.isfinite(arr)]
    arr = arr[arr >= 0]
    s = arr.sum()
    if s <= 0:
        return 0.0
    p = arr / s
    return float(np.sum(p * p))

# Local terms kept consistent with earlier text analysis
GERMAN_STATES = [
    "Baden-Wurttemberg", "Bavaria", "Berlin", "Brandenburg", "Bremen", "Hamburg", "Hesse",
    "Mecklenburg-Western Pomerania", "Lower Saxony", "North Rhine-Westphalia", "Rhineland-Palatinate",
    "Saarland", "Saxony", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"
]
CITIES_CORE = ["Berlin", "Hamburg", "Bielefeld", "Dresden", "Hildesheim", "Erfurt", "Offenbach", "Leipzig"]
DISTRICT_TO_CITY = {"Neukoelln": "Berlin"}  # extend as needed

LOCAL_TERMS = set(GERMAN_STATES) | set(CITIES_CORE) | set(DISTRICT_TO_CITY.keys())
TERM_REGEX = {t: re.compile(r"(?<![\wÄÖÜäöüß])" + re.escape(t) + r"(?![\wÄÖÜäöüß])", re.IGNORECASE)
              for t in LOCAL_TERMS}

## Describing the data and processing it for further analysis ##

In [ ]:
processed_data = {} # initialising empty dictionary to save processed DataFrames
region_names_dict = {} # to save region names in each dataset

In [ ]:
# Run the same functions on all files in the data_files

for name, file in data_files.items():

    print(f"### PROCESSING META AD DATA FOR {name.upper()} ###")
    print()

    df = pd.read_csv(file) # read the data file as a DataFrame

    print(f"This dataset contains {len(df)} ads from {len(df['page_name'].unique())} pages.")
    print()
    print("The pages included are:")
    print()
    print(df['page_name'].unique())
    print()

    df = add_average_spending(df)
    print()

    region_names, df_merged = process_delivery_by_region(df)
    print()

    region_names_dict[name] = region_names

    processed_data[name] = df_merged

    print(f"DataFrame saved as processed_data['{name}'].")
    print(f"Names of region columns added saved as region_names_dict['{name}'].")
    print()

    print("-------------------------------------")
    print()

## Further analysis ##

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# Define a single place for all generated artifacts
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

def get_spending_by_platform(name: str, df: pd.DataFrame, outdir: Path = OUT_DIR):
    """
    Calculate the total spend by platform and export a CSV.

    Parameters
    ----------
    name : str
        A short label for the campaign (used in the filename).
    df : pd.DataFrame
        Must contain columns: 'spend_average' and 'publisher_platforms'.
    outdir : Path
        Base directory for outputs (default: 'outputs').

    Returns
    -------
    spending_df : pd.DataFrame
    out_path : Path
    """
    # Basic checks
    required = {"spend_average", "publisher_platforms"}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"Missing columns in df: {missing}")

    # Filter by platform
    df_fb_insta = df[df["publisher_platforms"] == "facebook,instagram"]
    df_fb       = df[df["publisher_platforms"] == "facebook"]
    df_insta    = df[df["publisher_platforms"] == "instagram"]

    # Spending per platform
    fb_insta_spend = df_fb_insta["spend_average"].sum()
    fb_spend       = df_fb["spend_average"].sum()
    insta_spend    = df_insta["spend_average"].sum()
    total_spend    = fb_insta_spend + fb_spend + insta_spend

    # Status prints
    n = len(df)
    pct = lambda x: round(100 * x / n, 2) if n else 0.0
    print(f"{pct(len(df_fb_insta))}% of ads for {name.capitalize()}'s campaign ran on both Facebook and Instagram.")
    print(f"{pct(len(df_fb))}% only on Facebook, and {pct(len(df_insta))}% only on Instagram.")
    print(f"This added up to ${total_spend} in total.\n")

    # DataFrame
    spending_df = pd.DataFrame({
        "Platform": ["Facebook & Instagram", "Facebook", "Instagram", "Total"],
        "Total_Spend": [fb_insta_spend,       fb_spend,   insta_spend, total_spend]
    })

    # Save under a clean outputs structure
    platform_dir = outdir / "spending_by_platform"
    platform_dir.mkdir(parents=True, exist_ok=True)
    out_path = platform_dir / f"spending_by_platform_{name}.csv"
    spending_df.to_csv(out_path, index=False)
    print(f"Spending data exported to: {out_path}")

    return spending_df, out_path


In [ ]:
# ==== Estimate spending by state (portable) ====

from pathlib import Path
import re
import pandas as pd
import matplotlib.pyplot as plt

# single place for generated artifacts (typically gitignored)
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

def _ensure_spend_average(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure df has numeric 'spend_average'. If absent, derive midpoint from 'spend' ranges."""
    if "spend_average" in df.columns:
        return df
    if "spend" in df.columns:
        def _mid(s):
            if pd.isna(s):
                return 0.0
            nums = re.findall(r"\d+", str(s))
            if len(nums) >= 2:
                lo, hi = map(int, nums[:2])
                return (lo + hi) / 2.0
            elif len(nums) == 1:
                return float(nums[0])
            return 0.0
        df = df.copy()
        df["spend_average"] = df["spend"].apply(_mid)
        return df
    raise KeyError("Neither 'spend_average' nor 'spend' found in the DataFrame.")

def estimate_spending_by_state(
    name: str,
    df: pd.DataFrame,
    outdir: Path = OUT_DIR,
    plot: bool = True,
) -> tuple[pd.DataFrame, Path]:
    """
    Estimate ad spending per German state by multiplying spend_average by delivery percentages.

    Parameters
    ----------
    name : str
        Label for file output (e.g., 'die_linke').
    df : pd.DataFrame
        Must include spend column(s) and state percentage columns (0..100).
    outdir : Path
        Output base directory (default: 'outputs').
    plot : bool
        If True, show a bar chart.

    Returns
    -------
    state_spending_df : pd.DataFrame
        Columns: ['State', 'Estimated_Spend'] sorted desc.
    out_path : Path
        Path to the saved CSV.
    """
    # candidates incl. variants seen in your data
    state_candidates = [
        "Baden-Württemberg","Bayern","Berlin","Brandenburg","Bremen","Hamburg","Hessen",
        "Mecklenburg-Vorpommern","Niedersachsen","Nordrhein-Westfalen","Rheinland-Pfalz",
        "Saarland","Sachsen","Sachsen-Anhalt","Schleswig-Holstein","Thüringen",
        # extras seen in your dataset
        "Saxony-Anhalt","Berlin (city)","Bremen (city)","Alsace",
    ]

    df = _ensure_spend_average(df)

    # Keep only the state columns that actually exist
    available_states = [s for s in state_candidates if s in df.columns]
    if not available_states:
        raise ValueError(
            "No matching state columns were found in the DataFrame.\n"
            "Expected one or more of:\n  - " + ", ".join(state_candidates)
        )

    # Make sure state columns are numeric (coerce invalid to 0)
    df_num = df.copy()
    for s in available_states:
        df_num[s] = pd.to_numeric(df_num[s], errors="coerce").fillna(0.0)

    # Compute estimated spend per state
    state_spending = {
        s: float((df_num["spend_average"] * (df_num[s] / 100.0)).sum())
        for s in available_states
    }

    state_spending_df = (
        pd.DataFrame({"State": list(state_spending.keys()),
                      "Estimated_Spend": list(state_spending.values())})
        .sort_values("Estimated_Spend", ascending=False)
        .reset_index(drop=True)
    )

    # Save under a clean outputs structure
    state_dir = outdir / "spending_by_state"
    state_dir.mkdir(parents=True, exist_ok=True)
    out_path = state_dir / f"estimated_spending_by_state_{name}.csv"
    state_spending_df.to_csv(out_path, index=False)
    print(f"Saved: {out_path}")

    # Top-3 message
    top3 = state_spending_df.head(3)["State"].tolist()
    print(f"{name.capitalize()} spent the most in: {', '.join(top3)}")

    # Plot
    if plot:
        plt.figure(figsize=(10, 6))
        plt.bar(state_spending_df["State"], state_spending_df["Estimated_Spend"])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel("Estimated Spend (EUR)")
        plt.title(f"Estimated Spending by State — {name}")
        plt.tight_layout()
        plt.show()

    return state_spending_df, out_path

# Usage example (if you have df_with_regions already prepared):
# result_df, result_csv_path = estimate_spending_by_state("die_linke", df_with_regions)


In [ ]:
# ==== Merge + plot: Spending vs Votes (portable) ====

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Where we put generated artifacts by default
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

# ---- 1) Get spending-by-state ----
# Preferred: use the in-memory result from estimate_spending_by_state(...)
if "spending_by_state_df" in globals() and isinstance(spending_by_state_df, pd.DataFrame):
    _spending = spending_by_state_df.copy()
# Or maybe you stored it just as 'spending_df' with State/Estimated_Spend?
elif "spending_df" in globals() and isinstance(spending_df, pd.DataFrame) and \
     set(["State"]).issubset(spending_df.columns):
    _spending = spending_df.copy()
else:
    # Fallback: let the user paste a local CSV path (manual mode)
    SPENDING_CSV_PATH = r""  # e.g., r"C:\Users\Prof\Downloads\estimated_spending_by_state_die_linke.csv"
    if not SPENDING_CSV_PATH:
        raise ValueError(
            "No in-memory spending data found. Please either run the cell that creates "
            "spending_by_state_df or paste a local path into SPENDING_CSV_PATH."
        )
    _spending = pd.read_csv(Path(SPENDING_CSV_PATH))

# Harmonize column names for spending
if "Region" not in _spending.columns and "State" in _spending.columns:
    _spending = _spending.rename(columns={"State": "Region"})
if "Estimated_Spend" not in _spending.columns:
    # Common alternates:
    if "Total_Spend" in _spending.columns:
        _spending = _spending.rename(columns={"Total_Spend": "Estimated_Spend"})
    elif "Estimated_Spend_EUR" in _spending.columns:
        _spending = _spending.rename(columns={"Estimated_Spend_EUR": "Estimated_Spend"})
    else:
        raise KeyError(
            "Spending data is missing 'Estimated_Spend'. Found columns:\n"
            + ", ".join(_spending.columns)
        )

# ---- 2) Get votes by state ----
# Preferred: use in-memory 'votes_state' from load_state_votes_from_results_robust(...)
if "votes_state" in globals() and isinstance(votes_state, pd.DataFrame):
    _votes = votes_state.copy()
    # Build a 'Votes' column compatible with plotting (absolute Die Linke votes)
    if "Votes" not in _votes.columns:
        if "Votes_DieLinke" in _votes.columns:
            _votes["Votes"] = _votes["Votes_DieLinke"]
        elif "DieLinke_Zweitstimmen_Endgueltig" in _votes.columns:
            _votes = _votes.rename(columns={"DieLinke_Zweitstimmen_Endgueltig": "Votes"})
        else:
            raise KeyError(
                "Could not locate a Die Linke votes column in votes_state. "
                "Expected 'Votes_DieLinke' or 'DieLinke_Zweitstimmen_Endgueltig'."
            )
# Otherwise: manual path to a CSV that has Region + (Votes or DieLinke_Zweitstimmen_Endgueltig)
else:
    RESULTS_STATE_CSV_PATH = r""  # e.g., r"C:\Users\Prof\Downloads\die_linke_regions_2025.csv"
    if not RESULTS_STATE_CSV_PATH:
        raise ValueError(
            "No in-memory results found. Please either run the cell that creates 'votes_state' "
            "or paste a local path into RESULTS_STATE_CSV_PATH."
        )
    _votes = pd.read_csv(Path(RESULTS_STATE_CSV_PATH))

    # Harmonize column names for votes
    if "Region" not in _votes.columns:
        # Try a few common variants if needed
        for cand in ["Bundesland", "Land", "State"]:
            if cand in _votes.columns:
                _votes = _votes.rename(columns={cand: "Region"})
                break
    if "Votes" not in _votes.columns:
        if "DieLinke_Zweitstimmen_Endgueltig" in _votes.columns:
            _votes = _votes.rename(columns={"DieLinke_Zweitstimmen_Endgueltig": "Votes"})
        elif "Votes_DieLinke" in _votes.columns:
            _votes = _votes.rename(columns={"Votes_DieLinke": "Votes"})
        else:
            raise KeyError(
                "Could not find a Die Linke votes column. Expected one of "
                "['Votes', 'Votes_DieLinke', 'DieLinke_Zweitstimmen_Endgueltig']."
            )

# Minimal column checks
for df_, needed in [(_spending, {"Region", "Estimated_Spend"}), (_votes, {"Region", "Votes"})]:
    missing = needed - set(df_.columns)
    if missing:
        raise KeyError(f"Missing columns {missing} in dataframe with columns: {list(df_.columns)}")

# ---- 3) Merge ----
merged = pd.merge(_spending[["Region", "Estimated_Spend"]],
                  _votes[["Region", "Votes"]],
                  on="Region", how="inner")

# If you prefer vote share instead of raw votes:
# if "Votes_DieLinke" in _votes.columns and "Votes_Total" in _votes.columns:
#     merged = pd.merge(_spending[["Region", "Estimated_Spend"]],
#                       (_votes.assign(Vote_Share=_votes["Votes_DieLinke"] / _votes["Votes_Total"])
#                              [["Region", "Vote_Share"]]),
#                       on="Region", how="inner")
#     y_col = "Vote_Share"
# else:
y_col = "Votes"

# ---- 4) Plot ----
plt.figure(figsize=(8, 6))
sns.scatterplot(data=merged, x="Estimated_Spend", y=y_col, hue="Region", s=100)

# Regression line for overall trend (not per-region)
sns.regplot(data=merged, x="Estimated_Spend", y=y_col,
            scatter=False, color="black", line_kws={"linestyle": "--"})

plt.title("Ad Spending vs. Die Linke Votes (2025, by Bundesland)")
plt.xlabel("Estimated Spending (€)")
plt.ylabel("Die Linke Votes (Zweitstimme)" if y_col == "Votes" else "Die Linke Vote Share")
plt.tight_layout()
plt.show()

# ---- 5) Correlations ----
pearson_corr = merged["Estimated_Spend"].corr(merged[y_col], method="pearson")
spearman_corr = merged["Estimated_Spend"].corr(merged[y_col], method="spearman")

print("Correlation between spending and votes/share:")
print(f"  Pearson r = {pearson_corr:.3f}")
print(f"  Spearman ρ = {spearman_corr:.3f}")


In [ ]:
def drop_empty_ad_text(df):
    '''Removes all rows that do not have text in the 'ad_creative_bodies' column, for example video ads, for the purposes of text analysis.'''

    df_clean = df.dropna(subset=['ad_creative_bodies'], how='all')
    unique_ads_text = df_clean['ad_creative_bodies'].unique()

    print(f"Of the ads in the data, {len(df)-len(df_clean)} did not have any text accompanying the ads – for example video ads.")
    print(f"For textual analysis, we will only look at the remaining {len(df_clean)} ads that have text, including {len(unique_ads_text)} unique ones.")

    return df_clean

In [ ]:
# Initialise dictionary to store word combinations
top_bigrams = {}

In [ ]:
def normalise_unicode(text):
    '''Normalise any stylised Unicode text to standard alphabetic characters'''

    normalised_text = ''.join(
        unicodedata.normalize('NFKD', c)[0] if 'MATHEMATICAL' in unicodedata.name(c, '') else c
        for c in text
    )
    return normalised_text

In [ ]:
def preprocess_text(text):
    '''Remove times in formats like '4:00PM', '12:30AM', etc.'''

    text = re.sub(r'\b\d{1,2}:\d{2}\s?(AM|PM|am|pm)?\b', '', text)
    return text

In [ ]:
# Download stopwords, i.e. common words like 'the'
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))

In [ ]:
# Run the same functions to analyse all of the data files

for name, df in processed_data.items():

    print(f"### ANALYSING META AD DATA FOR {name.upper()} ###")
    print()

    spending_df = get_spending_by_platform(name, df)
    print()

    state_spending_df = estimate_spending_by_state(name, df)
    print()

    df_clean = drop_empty_ad_text(df)
    print()

    label_ctas(name, df_clean)
    print()

    get_most_frequent_phrases(name, df_clean)
    print()

    print("-------------------------------------")
    print()

In [ ]:
#demographic_distribution_analysis.py
import json
import os
import pandas as pd

csv_path = str(META_ADS_CSV)

df = pd.read_csv(csv_path)

def safe_json_loads(s):
    if not isinstance(s, str):
        return []
    # Replace single quotes with double quotes
    s = s.replace("'", '"')
    # Add brackets if missing
    if not s.strip().startswith("["):
        s = "[" + s + "]"
    try:
        return json.loads(s)
    except Exception as e:
        print(f"Error parsing: {s}\n{e}")
        return []

df["demographic_distribution"] = df["demographic_distribution"].apply(safe_json_loads)

df_exploded = df.explode("demographic_distribution")   

df_demographics = pd.concat([df_exploded.drop(columns=["demographic_distribution"]).reset_index(drop=True),pd.json_normalize(df_exploded["demographic_distribution"]).reset_index(drop=True)], axis=1)

print(df_demographics)

import matplotlib.pyplot as plt

# Group by age group and gender and compute the mean
pivot = df_demographics.groupby(["age", "gender"])['percentage'].mean().unstack()

# Sort age groups logically (optional, if needed)
# age_order = ["18-24", "25-34", "35-44", "45-54", "55-64", "65+"]
# pivot = pivot.reindex(age_order)

# Balkendiagramm zeichnen
pivot.plot(kind="barh")
plt.xlabel("Average share")
plt.ylabel("age group")
plt.title("Average demographic distribution by age group and gender")
plt.legend(title="gender")
plt.tight_layout()
plt.show()
# Save the processed DataFrame to a CSV file
output_csv_path = "data_for_vizzes/demographic_distribution.csv"


In [ ]:
# Relationship between spend and impressions
def extract_midpoint(value):
    if pd.isna(value):
        return None
    try:
        bounds = re.findall(r"\d+", value)
        if len(bounds) == 2:
            lower = int(bounds[0])
            upper = int(bounds[1])
            return (lower + upper) / 2
        elif len(bounds) == 1:
            return int(bounds[0])
    except:
        return None

# Midpoints berechnen
df["impressions_mid"] = df["impressions"].apply(extract_midpoint)
df["spend_mid"] = df["spend"].apply(extract_midpoint)

# Keep only valid rows
valid_rows = df.dropna(subset=["impressions_mid", "spend_mid"])

# Korrelation berechnen
correlation = valid_rows["impressions_mid"].corr(valid_rows["spend_mid"])
print(f"Korrelationskoeffizient zwischen Spend und Impressions: {correlation:.3f}")

# Scatter plot with log–log scale
if (valid_rows["spend_mid"] > 0).all() and (valid_rows["impressions_mid"] > 0).all():
    plt.figure(figsize=(8,6))
    plt.scatter(valid_rows["spend_mid"], valid_rows["impressions_mid"], alpha=0.5)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel("Spend (Midpoint EUR, log scale)")
    plt.ylabel("Impressions (Midpoint, log scale)")
    plt.title("Correlation between spend and impressions")
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    plt.show()
else:
    plt.figure(figsize=(8,6))
    plt.scatter(valid_rows["spend_mid"], valid_rows["impressions_mid"], alpha=0.5)
    plt.xlabel("Spend (Midpoint EUR)")
    plt.ylabel("Impressions (Midpoint)")
    plt.title("Correlation between spend and impressions")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Analyse voting data for the party "Die Linke" in the 2025 Bundestag elections
import pandas as pd

# Path to the file (adjust if needed)
vote_file = "/Users/karlahootz/Desktop/Jura/LLM/Bocconi-Kings College/Material Kings/Dissertation/case study/btw25_rws_bst2.csv"

# Read CSV, skip metadata rows, use ';' as delimiter and latin-1 encoding
votes_raw = pd.read_csv(vote_file, sep=";", skiprows=11, encoding="latin-1")

# Recode gender: w -> female, m|d|o -> male_other, Summe -> all
votes_raw["gender"] = votes_raw["Geschlecht"].replace({
    "w": "female",
    "m|d|o": "male_other",
    "Summe": "all"
})

# Keep only female and male_other
votes = votes_raw[votes_raw["gender"].isin(["female", "male_other"])].copy()

# Select only relevant columns (age, gender, votes for Die Linke, total)
votes = votes[["Geburtsjahresgruppe", "gender", "Die Linke", "Summe"]].rename(
    columns={"Geburtsjahresgruppe": "age_group", "Die Linke": "DIE LINKE"}
)

print(votes.head())


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- 1) Load election CSV (header after metadata) ---
vote_file = "/Users/karlahootz/Desktop/Jura/LLM/Bocconi-Kings College/Material Kings/Dissertation/case study/btw25_rws_bst2.csv"

votes_raw = pd.read_csv(vote_file, sep=";", skiprows=11, encoding="latin-1")

# Only second vote (party preference)
votes_raw = votes_raw[votes_raw["Erst-/Zweitstimme"] == 2].copy()

# Geschlecht harmonisieren
votes_raw["gender"] = votes_raw["Geschlecht"].replace({
    "w": "female",
    "m|d|o": "male_other",
    "Summe": "all"
})

# Age cohorts -> ad age buckets
age_map = {
    "2001-2007": "18-24",
    "1991-2000": "25-34",
    "1981-1990": "35-44",
    "1966-1980": "45-54",
    "1956-1965": "55-64",
    "<=1955": "65+",
    "≤1955": "65+"  # manche Dateien nutzen das ≤-Zeichen
}
votes_raw["age_group"] = votes_raw["Geburtsjahresgruppe"].map(age_map)

# Nur weiblich & male_other behalten
votes = votes_raw[votes_raw["gender"].isin(["female","male_other"])].copy()

# Stimmenanteil DIE LINKE je Alters×Geschlecht
# Note: column name must match your file exactly ("Die Linke")
if "Die Linke" not in votes.columns:
    raise KeyError("Spalte 'Die Linke' nicht gefunden. Prüfe die genaue Schreibweise der Parteispalte.")

votes_grp = (
    votes.groupby(["age_group","gender"], dropna=False)[["Die Linke","Summe"]]
    .sum()
    .reset_index()
)
votes_grp = votes_grp.dropna(subset=["age_group"])  # nur gemappte Altersgruppen
votes_grp["vote_share"] = votes_grp["Die Linke"] / votes_grp["Summe"]

# --- 2) Map ad impression shares to the same groups ---
# Expects df_demographics with columns: age (e.g., '18-24'), gender ('female','male','unknown'), percentage (0..1)
ads_long = df_demographics.copy()

# weiblich direkt
ads_female = (
    ads_long[ads_long["gender"] == "female"]
    .groupby("age", as_index=False)["percentage"]
    .mean()
    .rename(columns={"age":"age_group", "percentage":"ad_share_female"})
)

# male_other = male + unknown
ads_male = (
    ads_long[ads_long["gender"] == "male"]
    .groupby("age", as_index=False)["percentage"]
    .mean()
    .rename(columns={"age":"age_group", "percentage":"ad_share_male"})
)
ads_unknown = (
    ads_long[ads_long["gender"] == "unknown"]
    .groupby("age", as_index=False)["percentage"]
    .mean()
    .rename(columns={"age":"age_group", "percentage":"ad_share_unknown"})
)

ads_comb = (
    ads_female
    .merge(ads_male, on="age_group", how="outer")
    .merge(ads_unknown, on="age_group", how="outer")
    .fillna(0.0)
)
ads_comb["ad_share_male_other"] = ads_comb["ad_share_male"] + ads_comb["ad_share_unknown"]

# in Long-Form bringen (age_group, gender, ad_share)
ads_long2 = pd.concat([
    ads_comb[["age_group","ad_share_female"]].assign(gender="female").rename(columns={"ad_share_female":"ad_share"}),
    ads_comb[["age_group","ad_share_male_other"]].assign(gender="male_other").rename(columns={"ad_share_male_other":"ad_share"}),
], ignore_index=True)

# --- 3) Merge: Votes vs Ads ---
merged = votes_grp.merge(ads_long2, on=["age_group","gender"], how="inner")

from scipy.stats import pearsonr, spearmanr

# Drop NaNs just in case
df_corr = merged.dropna(subset=["vote_share", "ad_share"])

# Pearson correlation (linear)
pearson_r, pearson_p = pearsonr(df_corr["vote_share"], df_corr["ad_share"])

# Spearman correlation (rank-based, monotonic)
spearman_r, spearman_p = spearmanr(df_corr["vote_share"], df_corr["ad_share"])

print("Correlation between vote share (Die Linke) and ad impressions:")
print(f" Pearson r = {pearson_r:.3f} (p={pearson_p:.4f})")
print(f" Spearman ρ = {spearman_r:.3f} (p={spearman_p:.4f})")


# --- 4) Scatterplot ---
plt.figure(figsize=(7,7))
for g in merged["gender"].unique():
    sub = merged[merged["gender"] == g]
    plt.scatter(sub["vote_share"], sub["ad_share"], label=g, s=80)

# Diagonale y=x
low = min(merged["vote_share"].min(), merged["ad_share"].min())
high = max(merged["vote_share"].max(), merged["ad_share"].max())
plt.plot([low, high], [low, high], linestyle="--")

plt.xlabel("Vote share DIE LINKE (elections)")
plt.ylabel("Share of ad impressions (ads)")
plt.title("DIE LINKE: Share of votes vs. ad impressions by age × gender")
plt.legend(title="gender")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Optional: print the table
display_cols = ["age_group","gender","vote_share","ad_share","Die Linke","Summe"]
print(merged[display_cols].sort_values(["gender","age_group"]))


Robustness Check

In [ ]:
import pandas as pd
import numpy as np
import re

def load_state_votes_from_results_robust(path: str) -> pd.DataFrame:
    # Read raw lines and identify header rows
    with open(path, "r", encoding="latin-1") as f:
        lines = [ln.rstrip("\n") for ln in f.readlines()]

    # Assume 3 header rows at 5–7
    h1i, h2i, h3i = 5, 6, 7
    hdr1 = lines[h1i].split(";")
    hdr2 = lines[h2i].split(";")
    hdr3 = lines[h3i].split(";")
    maxlen = max(len(hdr1), len(hdr2), len(hdr3))
    def pad(row): return row + [""]*(maxlen - len(row))
    hdr1, hdr2, hdr3 = pad(hdr1), pad(hdr2), pad(hdr3)

    data_start = h3i + 1
    df = pd.read_csv(path, sep=";", header=None, skiprows=data_start,
                     encoding="latin-1", dtype=str)
    df = df.dropna(how="all")

    def htext(j):
        a, b, c = hdr1[j].strip().lower(), hdr2[j].strip().lower(), hdr3[j].strip().lower()
        return a, b, c, (a + " | " + b + " | " + c)

    region_idx, belongs_idx = None, None
    for j in range(maxlen):
        a, b, c, _ = htext(j)
        if region_idx is None and any(k in a+b+c for k in ["gebiet", "bundesland", "land"]):
            region_idx = j
        if belongs_idx is None and any(k in a+b+c for k in ["gehört", "gehoert", "geh. zu", "gehört zu"]):
            belongs_idx = j
    if region_idx is None: region_idx = 1
    if belongs_idx is None: belongs_idx = 2

    def is_zweit_count(j):
        a, b, c, _ = htext(j)
        txt = a + " " + b + " " + c
        if "zweit" not in txt:
            return False
        if any(k in txt for k in ["anteil", "quote", "%", "prozent"]):
            return False
        return True

    party_cols = {}
    tags = {
        "DIE LINKE": ["die linke", " linke "],
        "SPD": ["spd"], "CDU": ["cdu"], "GRUENE": ["grüne","gruene","bündnis 90","buendnis 90"],
        "FDP": ["fdp"], "AfD": ["afd"], "CSU": ["csu"],
        "BSW": ["bsw"], "FW": ["freie wähler","freie waehler","fw"]
    }
    for j in range(maxlen):
        if not is_zweit_count(j): continue
        _, _, _, full = htext(j)
        for party, needles in tags.items():
            if any(n in full for n in needles):
                party_cols.setdefault(party, []).append(j)

    def pick_best(idxs):
        if not idxs: return None
        ranked = []
        for j in idxs:
            _, _, _, full = htext(j)
            score = 0
            if "endg" in full: score += 2
            if "vorl" in full: score += 1
            ranked.append((score, j))
        ranked.sort(reverse=True)
        return ranked[0][1]

    party_first_idx = {p: pick_best(idxs) for p, idxs in party_cols.items() if idxs}
    if party_first_idx.get("DIE LINKE") is None:
        suspects = [(j, htext(j)[3]) for j in range(maxlen) if "zweit" in htext(j)[3] and "linke" in htext(j)[3]]
        raise KeyError("Could not locate DIE LINKE Zweitstimmen. Candidates:\n" +
                       "\n".join([f"[{j}] {txt}" for j, txt in suspects]))

    region = df.iloc[:, region_idx].astype(str).str.strip()
    belongs = df.iloc[:, belongs_idx].astype(str).str.strip()
    die_linke = pd.to_numeric(df.iloc[:, party_first_idx["DIE LINKE"]], errors="coerce")

    total_arrays = []
    for party, j in party_first_idx.items():
        if j is None: continue
        s = pd.to_numeric(df.iloc[:, j], errors="coerce")
        total_arrays.append(s)
    votes_total = sum(total_arrays) if total_arrays else die_linke.copy()

    mask_state = (belongs == "99")
    out = pd.DataFrame({
        "Region": region[mask_state],
        "Votes_DieLinke": die_linke[mask_state],
        "Votes_Total": votes_total[mask_state],
    }).dropna()

    out = out.groupby("Region", as_index=False).sum(numeric_only=True)
    return out


In [ ]:
# --- Spending data (already created by your earlier function) ---
spending_df = pd.read_csv(
    "outputs/spending_by_state/estimated_spending_by_state_die_linke.csv"
).rename(columns={"State": "Region"})

# --- Votes from results_2025_German_election.csv ---
votes_state = load_state_votes_from_results_robust(
    "/Users/karlahootz/Desktop/Jura/LLM/Bocconi-Kings College/Material Kings/Dissertation/case study/results_2025_German_election.csv"
)
votes_state["Vote_Share"] = votes_state["Votes_DieLinke"] / votes_state["Votes_Total"]

# --- Merge ---
merged_vote_share = spending_df.merge(
    votes_state[["Region", "Vote_Share"]], on="Region", how="inner"
)

# --- Plot + correlation ---
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import numpy as np

x = merged_vote_share["Estimated_Spend"].astype(float).values
y = merged_vote_share["Vote_Share"].astype(float).values

plt.figure(figsize=(7,6))
plt.scatter(x, y, s=80)
m, b = np.polyfit(x, y, 1)
plt.plot(np.linspace(min(x), max(x), 100), m*np.linspace(min(x), max(x), 100)+b, linestyle="--")
for xi, yi, lab in zip(x, y, merged_vote_share["Region"]):
    plt.text(xi, yi, lab, fontsize=8)
plt.xlabel("Estimated Meta Ad Spend (EUR)")
plt.ylabel("Die Linke Vote Share (Zweitstimme)")
plt.title("Spending vs Vote Share (Bundesländer)")
plt.tight_layout()
plt.show()

print("Pearson r:", pearsonr(x,y))
print("Spearman ρ:", spearmanr(x,y))
